<a href="https://colab.research.google.com/github/yk-Jeong/yk-jeong/blob/main/labq_%EC%9E%85%EC%82%AC%EA%B3%BC%EC%A0%9C_%EC%A0%95%EC%9C%A0%EA%B2%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**과제 목표**


- 테스트셋의 각 경주별로 마번 조합 3개를 예측
- 3개 조합 중 하나가 실제 복승 조합과 일치하면 적중
- 적중률 = 적중 경기수 / 총 경기수


**컬럼 설명** 
<pre>
rcDate 경주일자 rcNo 경주번호 chulNo 마번 hrNo 마 고유번호 age 마 연령 ageCond 연령조건
budam 부담구분 chaksunT 통산상금 chaksunY 최근1년상금 chaksun_6m 최근6개월상금 
dusu 출전 마 수 ilsu 경마일수 
ord1CntT 통산1위횟수 ord1CntY 최근1년1위횟수 ord2CntT  통산2위횟수 ord2CntY 최근1년2위횟수 ord3CntT  통산3위횟수 ord3CntY 최근1년3위횟수 
prd 마 산지 prizeCond 상금조건 rank 등급조건 rating 마 레이팅    
rcCntT 마 통산출전횟수 rcCntY 마 최근1년출전횟수 
rcDist 경주거리 sex 마 성별 wgBudam 부담중량 
track 경마장 상태 weather 날씨 waterRate 경마장 함수율
ord 등수 rcTime 경주기록(초) 
jkNo 기수번호 trNo 조교사번호 owNo 마주번호
</pre>

###1. 데이터 확인

라이브러리 로드 

In [286]:
import pandas as pd
import numpy as np

데이터 로드 

In [287]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission = pd.read_csv('sample_submission.csv')

데이터 개형 확인

In [288]:
df_train

,rcDate,rcNo,chulNo,hrNo,age,ageCond,budam,chaksunT,chaksunY,chaksun_6m,...,sex,wgBudam,track,weather,waterRate,ord,rcTime,jkNo,trNo,owNo
0,20210108,1,1,43108,3,연령오픈,별정A,0,0,0,...,수,56.5,건조,맑음,3,11,66.1,80438,70128,118030
1,20210108,1,2,42271,3,연령오픈,별정A,0,0,0,...,거,52.5,건조,맑음,3,12,68.4,80601,70086,104133
2,20210108,1,3,42905,3,연령오픈,별정A,0,0,0,...,수,56.5,건조,맑음,3,9,65.9,80588,70129,112286
3,20210108,1,4,42445,3,연령오픈,별정A,0,0,0,...,암,54.5,건조,맑음,3,7,65.2,80532,70135,112274
4,20210108,1,5,42583,3,연령오픈,별정A,0,0,0,...,거,56.5,건조,맑음,3,6,64.8,80421,70137,110050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7059,20211226,7,7,41710,5,연령오픈,핸디캡,475550000,100000000,46800000,...,수,51.5,건조,흐림,3,10,102.2,80606,70186,106092
7060,20211226,7,8,41804,5,연령오픈,핸디캡,297200000,77900000,71300000,...,수,51.0,건조,흐림,3,5,99.8,80456,70141,104123
7061,20211226,7,9,43191,3,연령오픈,핸디캡,232710000,193950000,144350000,...,수,51.0,건조,흐림,3,6,100.0,80423,70086,201269
7062,20211226,7,10,40310,4,연령오픈,핸디캡,249440000,128100000,94600000,...,거,52.0,건조,흐림,3,4,99.8,80438,70121,112274


In [289]:
# ord(등수), rcTime(경주기록(초)) 컬럼이 없는 test file
df_test

,rcDate,rcNo,chulNo,hrNo,age,ageCond,budam,chaksunT,chaksunY,chaksun_6m,...,rcCntY,rcDist,sex,wgBudam,track,weather,waterRate,jkNo,trNo,owNo
0,20220107,1,1,44563,3,3세,별정A,0,0,0,...,5,1000,암,54.0,건조,맑음,3,80580,70129,109149
1,20220107,1,2,44911,3,3세,별정A,2400000,2400000,2400000,...,2,1000,암,54.0,건조,맑음,3,80423,70130,103007
2,20220107,1,3,44180,3,3세,별정A,0,0,0,...,0,1000,암,52.0,건조,맑음,3,80600,70180,104145
3,20220107,1,4,45039,3,3세,별정A,13200000,13200000,13200000,...,3,1000,암,54.0,건조,맑음,3,80456,70173,104127
4,20220107,1,5,43816,3,3세,별정A,0,0,0,...,0,1000,암,54.0,건조,맑음,3,80546,70136,117054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,20220130,6,9,43191,4,연령오픈,핸디캡,232710000,185150000,130350000,...,12,1400,수,52.0,건조,맑음,2,80484,70086,201269
650,20220130,6,10,38065,8,연령오픈,핸디캡,462550000,28500000,0,...,6,1400,거,52.0,건조,맑음,2,80434,70128,113115
651,20220130,6,11,38180,8,연령오픈,핸디캡,358200000,0,0,...,2,1400,암,51.0,건조,맑음,2,80482,70141,104131
652,20220130,6,13,36510,9,연령오픈,핸디캡,1470900000,19800000,19800000,...,6,1400,수,55.5,건조,맑음,2,80600,70136,105040


In [290]:
df_test['rcDate'].unique()

array([20220107, 20220109, 20220114, 20220116, 20220121, 20220123,
       20220128, 20220130])

In [291]:
# 예측 예시는 다음과 같이 경주일자 - 경주번호 - 복승식*3 으로 추출하여야 함
df_submission

,rcDate,rcNo,pred1,pred2,pred3
0,20220130,1,3-5,3-7,5-7


In [292]:
# 결측치 여부 등 기본적인 데이터 정보 확인 
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7064 entries, 0 to 7063
Data columns (total 35 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rcDate      7064 non-null   int64  
 1   rcNo        7064 non-null   int64  
 2   chulNo      7064 non-null   int64  
 3   hrNo        7064 non-null   int64  
 4   age         7064 non-null   int64  
 5   ageCond     7064 non-null   object 
 6   budam       7064 non-null   object 
 7   chaksunT    7064 non-null   int64  
 8   chaksunY    7064 non-null   int64  
 9   chaksun_6m  7064 non-null   int64  
 10  dusu        7064 non-null   int64  
 11  ilsu        7064 non-null   int64  
 12  ord1CntT    7064 non-null   int64  
 13  ord1CntY    7064 non-null   int64  
 14  ord2CntT    7064 non-null   int64  
 15  ord2CntY    7064 non-null   int64  
 16  ord3CntT    7064 non-null   int64  
 17  ord3CntY    7064 non-null   int64  
 18  prd         7064 non-null   object 
 19  prizeCond   7064 non-null  

###2. 특성공학

1. 컬럼명을 한글로 변경 

In [293]:
# 컬럼명을 보기 편하게 변경
df_train.columns = ['경주일자', '경주번호', '마번', '고유번호', '연령', '연령조건', '부담구분', '통산상금', '최근 1년 상금', '최근 6개월 상금', '출전 마 수', '경마일수', '통산 1위 횟수', '최근 1년 1위 횟수', '통산 2위 횟수', '최근 1년 2위 횟수', '통산 3위 횟수', '최근 1년 3위 횟수', '산지', '상금조건', '등급조건', '마 레이팅', '통산출전횟수', '최근 1년 출전 횟수', '경주거리', '마 성별', '부담중량', '경마장 상태', '날씨', '경마장 함수율', '등수', '경주기록(초)', '기수번호', '조교사 번호', '마주번호']
df_test.columns = ['경주일자', '경주번호', '마번', '고유번호', '연령', '연령조건', '부담구분', '통산상금', '최근 1년 상금', '최근 6개월 상금', '출전 마 수', '경마일수', '통산 1위 횟수', '최근 1년 1위 횟수', '통산 2위 횟수', '최근 1년 2위 횟수', '통산 3위 횟수', '최근 1년 3위 횟수', '산지', '상금조건', '등급조건', '마 레이팅', '통산출전횟수', '최근 1년 출전 횟수', '경주거리', '마 성별', '부담중량', '경마장 상태', '날씨', '경마장 함수율', '기수번호', '조교사 번호', '마주번호']

2. 레이팅 값이 -인 경우를 결측치로 해석, train과 test 각각의 평균값으로 대체

In [294]:
df_train['마 레이팅'] = df_train['마 레이팅'].replace('-', '75', regex=True)
df_test['마 레이팅'] = df_test['마 레이팅'].replace('-', '61', regex=True)

3. 형변환: numeric이지만 실제로는 고유한 값인 컬럼들의 형을 object로 변경

  `경주일자, 경주번호, 마번, 고유번호, 기수번호, 조교사 번호, 마주번호` -> str

In [295]:
df_train = df_train.astype({'마 레이팅':'int', '경주일자':'str', '경주번호':'str', '마번':'str', '고유번호':'str', '기수번호':'str', '조교사 번호':'str', '마주번호':'str'})
# test set도 같은 처리를 해줌 
df_test = df_test.astype({'마 레이팅':'int', '경주일자':'str', '경주번호':'str', '마번':'str', '고유번호':'str', '기수번호':'str', '조교사 번호':'str', '마주번호':'str'})

In [296]:
# 의도대로 형변환되었는지 확인
df_train.info(), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7064 entries, 0 to 7063
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   경주일자         7064 non-null   object 
 1   경주번호         7064 non-null   object 
 2   마번           7064 non-null   object 
 3   고유번호         7064 non-null   object 
 4   연령           7064 non-null   int64  
 5   연령조건         7064 non-null   object 
 6   부담구분         7064 non-null   object 
 7   통산상금         7064 non-null   int64  
 8   최근 1년 상금     7064 non-null   int64  
 9   최근 6개월 상금    7064 non-null   int64  
 10  출전 마 수       7064 non-null   int64  
 11  경마일수         7064 non-null   int64  
 12  통산 1위 횟수     7064 non-null   int64  
 13  최근 1년 1위 횟수  7064 non-null   int64  
 14  통산 2위 횟수     7064 non-null   int64  
 15  최근 1년 2위 횟수  7064 non-null   int64  
 16  통산 3위 횟수     7064 non-null   int64  
 17  최근 1년 3위 횟수  7064 non-null   int64  
 18  산지           7064 non-null   object 
 19  상금조건  

(None, None)

3. 분류화
  - 복승식 예측이므로 1위부터 2위까지의 말을 1, 나머지를 0으로 치환하여 **분류** 문제로 해결하고자 함

In [297]:
df_train['승리'] = np.where((df_train['등수'].values <= 2), 1, 0)

4. 컬럼 삭제 및 분리
- target 컬럼을 포함해 train set의 feature 수가 test set보다 1개 많도록 하여야 함 
- 차원수 감소를 위해 상대적으로 관련도가 낮다고 판단되는 feature를 분리
- 경주일자 feature를 연, 월, 일로 분리하였으므로 원본 컬럼은 삭제 
<pre>
상금조건, 등급조건, 통산상금, 최근1년상금, 최근6개월상금, 출전 마 수
</pre>



In [298]:
train_sep = df_train[['상금조건', '등급조건', '통산상금', '최근 1년 상금', '최근 6개월 상금', '출전 마 수']]
test_sep = df_test[['상금조건', '등급조건', '통산상금', '최근 1년 상금', '최근 6개월 상금', '출전 마 수']]

In [299]:
df_train.drop(labels = ['상금조건', '등급조건', '경주기록(초)','등수', '통산상금', '최근 1년 상금', '최근 6개월 상금', '출전 마 수'], axis = 1, inplace = True)
df_test.drop(labels = ['상금조건', '등급조건', '통산상금', '최근 1년 상금', '최근 6개월 상금', '출전 마 수'], axis = 1, inplace = True)

In [300]:
# train set의 feature 수가 test set보다 1개 많은지 확인
df_train.shape, df_test.shape

((7064, 28), (654, 27))

핸들링이 끝났다면 원본을 남겨두고 복제

In [301]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7064 entries, 0 to 7063
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   경주일자         7064 non-null   object 
 1   경주번호         7064 non-null   object 
 2   마번           7064 non-null   object 
 3   고유번호         7064 non-null   object 
 4   연령           7064 non-null   int64  
 5   연령조건         7064 non-null   object 
 6   부담구분         7064 non-null   object 
 7   경마일수         7064 non-null   int64  
 8   통산 1위 횟수     7064 non-null   int64  
 9   최근 1년 1위 횟수  7064 non-null   int64  
 10  통산 2위 횟수     7064 non-null   int64  
 11  최근 1년 2위 횟수  7064 non-null   int64  
 12  통산 3위 횟수     7064 non-null   int64  
 13  최근 1년 3위 횟수  7064 non-null   int64  
 14  산지           7064 non-null   object 
 15  마 레이팅        7064 non-null   int64  
 16  통산출전횟수       7064 non-null   int64  
 17  최근 1년 출전 횟수  7064 non-null   int64  
 18  경주거리         7064 non-null   int64  
 19  마 성별  

In [302]:
train = df_train.copy()
test = df_test.copy()

###3. 모델링

0. 타겟 설정 & random seed 고정

- `target = '승리'`로 target을 설정
- 난수: `42`로 고정

In [303]:
target = '승리'

In [304]:
# 난수 고정
randnum = 42

1. train set를 train / validation으로 분할
- 분할비율은 train:val = 8:2로 임의 지정


In [305]:
from sklearn.model_selection import train_test_split

In [306]:
train, val = train_test_split(train, test_size = 0.2, random_state = randnum)

# 훈련 / 검증 데이터로 분할한 후 target column 별도 분할
X_train = train.drop(columns = target)
y_train = train[target]
X_val = val.drop(columns = target)
y_val = val[target]
X_test = test

In [307]:
# 훈련 / 검증 데이터 각각의 개형 확인 

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape

((5651, 27), (5651,), (1413, 27), (1413,), (654, 27))

In [308]:
# target값의 분포를 확인
y_train.value_counts(normalize=True)

0    0.83118
1    0.16882
Name: 승리, dtype: float64

약 83:17으로 비율이 상당히 불균형하나 우선은 별도의 처리 없이 모델에 적합하기로 결정

2. encoding 
- 순서형 변수가 대부분이며, XGboost를 사용할 것이므로 인코더로 **ordinal encoder**를 채택

In [309]:
# encoder 설치 및 로드 
!pip install --upgrade category_encoders
from category_encoders import OrdinalEncoder
encoder = OrdinalEncoder()

# 데이터에 인코딩 적용 
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


4. 모델 적합
- Gradient Boosting 기법을 사용하고자 **XGboost lib** 채택 
- 과적합을 방지하고 하이퍼파라미터를 최적화하고자 **early stopping** 적용

In [310]:
!pip3 install xgboost
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [311]:
model = XGBClassifier(
        objective = "binary:logistic",
        eval_metric = "error", 
        n_estimators = 1000000,
        random_state = randnum,
        n_jobs = -1,
        max_depth = 5,
        learning_rate = 0.1)

# 기준 dataset 설정
watch_list = [(X_train_encoded, y_train), (X_val_encoded, y_val)]

# 모델 학습 - early stopping 시점은 round 100까지로 임의 설정
model.fit(X_train_encoded, y_train, eval_set = watch_list, early_stopping_rounds = 100)

[0]	validation_0-error:0.155725	validation_1-error:0.171267
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.154663	validation_1-error:0.166313
[2]	validation_0-error:0.154132	validation_1-error:0.166313
[3]	validation_0-error:0.155017	validation_1-error:0.163482
[4]	validation_0-error:0.152893	validation_1-error:0.166313
[5]	validation_0-error:0.152185	validation_1-error:0.163482
[6]	validation_0-error:0.152185	validation_1-error:0.160651
[7]	validation_0-error:0.151478	validation_1-error:0.160651
[8]	validation_0-error:0.150416	validation_1-error:0.15782
[9]	validation_0-error:0.150062	validation_1-error:0.160651
[10]	validation_0-error:0.149177	validation_1-error:0.158528
[11]	validation_0-error:0.147761	validation_1-error:0.158528
[12]	validation_0-error:0.148115	validation_1-error:0.158528
[13]	validation_0-error:0.147054	validation_1-error:0.15

XGBClassifier(eval_metric='error', max_depth=5, n_estimators=1000000, n_jobs=-1,
              random_state=42)

#### 4. 모델 검증
- 분류모델이므로 평가지표를 **accuracy**로 설정해 검증

In [312]:
from sklearn.metrics import classification_report

print(
    classification_report(
        y_val, model.predict(X_val_encoded)
    )
)
print("validation accuracy", model.score(X_val_encoded, y_val))

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      1191
           1       0.60      0.11      0.19       222

    accuracy                           0.85      1413
   macro avg       0.73      0.55      0.55      1413
weighted avg       0.82      0.85      0.80      1413

validation accuracy 0.8485491861288039


### 5. 결과 확인

모델을 test dataset에 적용

In [313]:
X_test_encoded = encoder.transform(X_test) 
output = model.predict(X_test_encoded)

결과값을 원본 test 데이터와 병합 후 승리(1)로 예측된 마번을 추출

In [314]:
output = pd.DataFrame(output)
output_submission = pd.concat([X_test, output], axis=1)
output_submission = output_submission.loc[(output_submission[0] == 1)]

In [315]:
# 필요한 컬럼만 출력해 데이터 형태 확인

df_submission = output_submission[['경주일자', '경주번호', '마번']]
df_submission

,경주일자,경주번호,마번
12,20220107,2,6
76,20220107,7,10
80,20220107,8,2
162,20220114,1,2
202,20220114,4,9
224,20220114,6,8
231,20220114,7,7
235,20220114,8,3
242,20220114,8,10
270,20220116,3,2


결과를 .csv 파일로 추출

In [316]:
df_submission.to_csv('df_submission.csv', mode = 'w')

### 해결하지 못한 문제

- 특성공학 과정에서의 컬럼 추가 시도 실패(`속도`, `백분율`)
  - 경기에 따라 말이 달려야 하는 거리가 다르므로, 경주거리를 고려하지 않고 경주기록을 단순 비교할 수는 없음 
  - 마찬가지로 각 경주에 참여한 전체 말의 수를 고려하여 등수를 상대화한 뒤 비교하여야 함 
  - 그러나 test set에 `등수`와 `경주기록(초)` 컬럼이 존재하지 않아 train set에도 적용할 수 없었음 

In [317]:
# velocity 
# df_train['속도'] = df_train['경주거리'] / df_train['경주기록(초)'] 
# percentile
# df_train['백분율'] = (df_train['등수'] / df_train['출전 마 수']) * 100 

- domain knowledge의 부족으로 차원수 축소를 위한 feature 삭제 과정에서 충분한 근거가 확보되지 못함 
- 시간관계상 target feature의 불균형 문제를 해소하지 않고 모델링을 진행함 -> 모든 회차의 경기에서 3쌍씩을 추출할 수 없었음